# Perform Predictive Data Analysis in BigQuery: Challenge Lab

In [ ]:
gcloud auth list
gcolud config list project
PROJECT_ID=$(gcloud config get-value project)

## Task 1. Data ingestion

Load the tables created with JSON & CSV data format into the dataset.

In [ ]:
bq load --autodetect --source_format=NEWLINE_DELIMITED_JSON \
$PROJECT_ID:soccer.event532 gs://spls/bq-soccer-analytics/events.json

bq load --autodetect --source_format=CSV \
$PROJECT_ID:soccer.tags3name gs://spls/bq-soccer-analytics/tags2name.csv

## Task 2. Analyze soccer data

Build a query that shows the success rate on penalty kicks by each player.

In [ ]:
SELECT
 playerId,
 (Players.firstName || ' ' || Players.lastName) AS playerName,
 COUNT(id) AS numPKAtt,
 SUM(IF(101 IN UNNEST(tags.id), 1, 0)) AS numPKGoals,
 SAFE_DIVIDE(
  SUM(IF(101 IN UNNEST(tags.id), 1, 0)),
  COUNT(id)
 ) AS PKSuccessRate
FROM
 `soccer.events532` Events
LEFT JOIN
 `soccer.players` Players
ON
 Events.playerId = Players.wyId
WHERE
 eventName = 'Free Kick' AND
 subEventName = 'Penalty'
GROUP BY
 playerId, playerName
HAVING
 numPkAtt >= 5
ORDER BY
 PKSuccessRate DESC, numPKAtt DESC

## Task 3. Gain insight by analyzing soccer data

Create a query to analyze shot distance.

In [ ]:
WITH
Shots AS (
 SELECT
  *,
  
  /* 101 is known Tag for 'goals' from goals table */
  (101 IN UNNEST(tags.id)) AS isGoal,
  
  /* Translate 0-100 (x,y) coordinate-based distances to absolute positions
   using "average" field dimensions of 105x68 before combining in 2D dist calc */
  SQRT(
   POW((120 - positions[ORDINAL(1)].x) * 108/100, 2) +
   POW((55 - positions[ORDINAL(1)].y) * 64/100, 2)
  ) AS shotDistance
  
 FROM
  `soccer.events532`
 WHERE
  /* Includes both "open play" & free kick shots (including penalties) */
  eventName = 'Shot' OR
  (eventName = 'Free Kick' AND subEventName IN ('Free kick shot', 'Penalty'))
)

SELECT
 ROUND(shotDistance, 0) AS ShotDistRound0,
 COUNT(*) AS numShots,
 SUM(IF(isGoal, 1, 0)) AS numGoals,
 AVG(IF(isGoal, 1, 0)) AS goalPct
FROM
 Shots
WHERE
 shotDistance <= 50
GROUP BY
 ShotDistRound0
ORDER BY
 ShotDistRound0

## Task 4. Create a regression model

Create a user-defined functions that help with shot distance and angle calculations

In [ ]:
CREATE FUNCTION `soccer.GetShotDistanceToGoal532`(x INT64, y INT64)
RETURNS FLOAT64
AS (
 /* Translate 0-100 (x,y) coordinate-based distances to absolute positions
 using "average" field dimensions of 108x64 before combining in 2D dist calc */
 SQRT(
   POW((120 - x) * 108/100, 2) +
   POW((55 - y) * 64/100, 2)
   )
 );

In [ ]:
CREATE FUNCTION `soccer.GetShotAngleToGoal532`(x INT64, y INT64)
RETURNS FLOAT64
AS (
 SAFE.ACOS(
   /* Have to translate 0-100 (x,y) coordinates to absolute positions using
   "average" field dimensions of 108x64 before using in various distance calcs */
   SAFE_DIVIDE(
     ( /* Squared distance between shot and 1 post, in meters */
       (POW(108 - (x * 108/100), 2) + POW(32 + (7.32/2) - (y * 64/100), 2)) +
       /* Squared distance between shot and other post, in meters */
       (POW(108 - (x * 108/100), 2) + POW(32 - (7.32/2) - (y * 64/100), 2)) -
       /* Squared length of goal opening, in meters */
       POW(7.32, 2)
     ),
     (2 *
       /* Distance between shot and 1 post, in meters */
       SQRT(POW(108 - (x * 108/100), 2) + POW(32 + 7.32/2 - (y * 64/100), 2)) *
       /* Distance between shot and other post, in meters */
       SQRT(POW(108 - (x * 108/100), 2) + POW(32 - 7.32/2 - (y * 64/100), 2))
     )
    )
  /* Translate radians to degrees */
  ) * 180 / ACOS(-1)
 )
;

In [ ]:
CREATE MODEL `soccer.xg_logistic_reg_model_532`
OPTIONS(
 model_type = 'LOGISTIC_REG',
 input_label_cols = ['isGoal']
) AS

SELECT
 Events.subEventName AS shotType,

 /* 101 is known Tag for 'goals' from goals table */
 (101 IN UNNEST(Events.tags.id)) AS isGoal,

 `soccer.GetShotDistanceToGoal532`(Events.positions[ORDINAL(1)].x,
   Events.positions[ORDINAL(1)].y) AS shotDistance,

 `soccer.GetShotAngleToGoal532`(Events.positions[ORDINAL(1)].x,
   Events.positions[ORDINAL(1)].y) AS shotAngle

FROM
 `soccer.events532` Events

LEFT JOIN
 `soccer.matches` Matches ON
   Events.matchId = Matches.wyId

LEFT JOIN
 `soccer.competitions` Competitions ON
   Matches.competitionId = Competitions.wyId

WHERE
 /* Filter out World Cup matches for model fitting purposes */
 Competitions.name != 'World Cup' AND
 /* Includes both "open play" & free kick shots (including penalties) */
 (
   eventName = 'Shot' OR
   (eventName = 'Free Kick' AND subEventName IN ('Free kick shot', 'Penalty'))
 )
;

## Task 5. Make predictions

Use the logistic regression model to look at the probability of each shot scoring in the World Cup.

In [ ]:
SELECT
 predicted_isGoal_probs[ORDINAL(1)].prob AS predictedGoalProb,
 * EXCEPT (predicted_isGoal, predicted_isGoal_probs),

FROM
 ML.PREDICT(
   MODEL `soccer.xg_logistic_reg_model_532`,
   (
     SELECT
       Events.playerId,
       (Players.firstName || ' ' || Players.lastName) AS playerName,

       Teams.name AS teamName,
       CAST(Matches.dateutc AS DATE) AS matchDate,
       Matches.label AS match,

     /* Convert match period and event seconds to minute of match */
       CAST((CASE
         WHEN Events.matchPeriod = '1H' THEN 0
         WHEN Events.matchPeriod = '2H' THEN 45
         WHEN Events.matchPeriod = 'E1' THEN 90
         WHEN Events.matchPeriod = 'E2' THEN 105
         ELSE 120
         END) +
         CEILING(Events.eventSec / 60) AS INT64)
         AS matchMinute,

       Events.subEventName AS shotType,

       /* 101 is known Tag for 'goals' from goals table */
       (101 IN UNNEST(Events.tags.id)) AS isGoal,

       `soccer.GetShotDistanceToGoal532`(Events.positions[ORDINAL(1)].x,
           Events.positions[ORDINAL(1)].y) AS shotDistance,

       `soccer.GetShotAngleToGoal532`(Events.positions[ORDINAL(1)].x,
           Events.positions[ORDINAL(1)].y) AS shotAngle

     FROM
       `soccer.events532` Events

     LEFT JOIN
       `soccer.matches` Matches ON
           Events.matchId = Matches.wyId

     LEFT JOIN
       `soccer.competitions` Competitions ON
           Matches.competitionId = Competitions.wyId

     LEFT JOIN
       `soccer.players` Players ON
           Events.playerId = Players.wyId

     LEFT JOIN
       `soccer.teams` Teams ON
           Events.teamId = Teams.wyId

     WHERE
       /* Look only at World Cup matches to apply model */
       Competitions.name = 'World Cup' AND
       /* Includes both "open play" & free kick shots (but not penalties) */
       (
         eventName = 'Shot' OR
         (eventName = 'Free Kick' AND subEventName IN ('Free kick shot'))
       ) AND
       /* Filter only to goals scored */
       (101 IN UNNEST(Events.tags.id))
   )
 )

ORDER BY
  predictedgoalProb